# 인터파크 베스트 셀러 도서 정보

In [2]:
import requests 
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req = requests.get(url)

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(req.text, 'html.parser')

### Step 1. 찾고자 하는 목록들을 가져온다.(list 계열의 li, table계열의 tr)



In [4]:
# 화면 내의 모든 li 태그를 찾아주니까 이렇게 하면 안됨
lis = soup.find_all('li') #li : list item
len(lis)

3532

In [6]:
# 15개 베스트셀러 목록의 li 태그
lis = soup.select_one('.rankBestContentList').find_all('li')
len(lis)

58

In [8]:
# ol 태그의 자식 태그 li 만 찾아주는 방법
lis = soup.select_one('.rankBestContentList').select('ol > li')
len(lis)

15

In [9]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

### Step 2. 목록들 중에 하나를 선택해 세부 내용을 가져온다.

In [10]:
li = lis[0]

In [12]:
# 책 제목
title = li.select_one('.itemName').get_text().strip() #맨 뒤에 공백 하나 붙어서 strip 사용 
title

'하얼빈'

In [13]:
# 저자
author = li.select_one('.author').get_text().strip()
author

'김훈(金薰)'

In [14]:
# 출판사
company = li.select_one('.company').get_text().strip()
company

'문학동네'

In [16]:
# 가격
price = li.select_one('.price > em').get_text().strip() #price에서 가격만 적힌 em태그를 가져오겠다. 
price = int(price.replace(',','')) # 금액부분에서 ,를 아무것도 아닌걸로 바꾸겠다.(,를 빼겠다.) #그리고 int로 
price

14400

In [25]:
# 순위 (한자리수 순위와 두자리수 순위를 둘다 고려해야함)
#지금은 14위 순위 가지고 오기
li = lis[14]
rank_spans = li.select('.rankBtn_ctrl')
rank_spans

[<span class="rankBtn_ctrl rkNum_M01 "></span>,
 <span class="rankBtn_ctrl rkNum_M04 "></span>]

In [26]:
for span in rank_spans:
  print(span['class'])

['rankBtn_ctrl', 'rkNum_M01', '']
['rankBtn_ctrl', 'rkNum_M04', '']


In [27]:
for span in rank_spans:
  print(span['class'][1][-1]) #rkNum의 맨 마지막 숫자를 가지고 온 것 = 순위

1
4


In [29]:
rank = ''  #string
for span in rank_spans:  #list
  rank += span['class'][1][-1]
rank = int(rank)
rank

14

### Step 3. 모든 목록에 대해 세부 정보를 가져와서 데이터프레임으로 만든다. 

In [30]:
import pandas as pd

In [35]:
lines = []
for li in lis:
  title = li.select_one('.itemName').get_text().strip()
  author = li.select_one('.author').get_text().strip()
  company = li.select_one('.company').get_text().strip()
  price = li.select_one('.price > em').get_text().strip()
  price = int(price.replace(',','')) 
  rank = ''  
  rank_spans = li.select('.rankBtn_ctrl')
  for span in rank_spans:  
    rank += span['class'][1][-1]
  rank = int(rank)
  lines.append([rank,title,author,company,price]) #리스트 안에 리스트가 생기는 2차원 구조 

In [38]:
# 어디서 에러가 발생했는지 알고 싶은 경우
lines = []
for i, li in enumerate(lis):
  try:
      title = li.select_one('.itemName').get_text().strip()
      author = li.select_one('.author').get_text().strip()
      company = li.select_one('.company').get_text().strip()
      price = li.select_one('.price > em').get_text().strip()
      price = int(price.replace(',','')) 
      rank = ''  
      rank_spans = li.select('.rankBtn_ctrl')
      for span in rank_spans:  
        rank += span['class'][1][-1]
      rank = int(rank)
      lines.append([rank,title,author,company,price]) #리스트 안에 리스트가 생기는 2차원 구조 
  except:
    print(i)

In [39]:
df = pd.DataFrame(lines, columns=['순위','제목','저자','출판사','가격'])
df

,순위,제목,저자,출판사,가격
0,1,하얼빈,김훈(金薰),문학동네,14400
1,2,미래의 부를 위한 투자 공부,신진상,미디어숲,16020
2,3,불편한 편의점 2,김호연,나무옆의자,12600
3,4,계속 가보겠습니다,임은정,메디치미디어,16200
4,4,불편한 편의점(40만부 기념 벚꽃 에디션),김호연,나무옆의자,12600
5,6,가불 선진국,조국,메디치미디어,14400
6,7,헤어질 결심 각본,"정서경, 박찬욱",을유문화사,13500
7,8,파친코 1,이민진/신승미 역,인플루엔셜,14220
8,8,아직 긴 인생이 남았습니다,기시미 이치로(岸見一郞)/전경아 역,한국경제신문,14400
9,10,마법천자문 53: 결정을 내리다! 결단할 결,유대영/홍거북 그림/김창환 감수,아울북,12600
